In [39]:
import pandas as pd
import numpy as np
import datetime
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics

In [25]:
spark = SparkSession.builder.getOrCreate()

In [6]:
ratings = pd.read_csv('./data/ratings.csv')

In [7]:
ratings['timestamp'] = ratings['timestamp'].apply(datetime.datetime.fromtimestamp)
sample = ratings[ratings['timestamp'].apply(lambda x: x.date()) >= datetime.date(2013,3,31)]

In [26]:
df = spark.createDataFrame(sample)
model = ALS(userCol='userId', 
            itemCol='movieId', 
            ratingCol='rating').fit(df)

In [29]:
predictions = model.transform(df)
predictions.toPandas().head()

,userId,movieId,rating,timestamp,prediction
0,36445,148,4.5,2014-12-23 13:15:55,4.364294
1,3990,148,4.0,2015-02-01 14:04:54,3.808037
2,28397,148,5.0,2013-04-24 17:58:49,4.738766
3,29100,463,1.0,2015-03-14 11:36:32,1.115891
4,26855,463,2.0,2013-05-11 00:26:07,1.978067


In [38]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
evaluator.evaluate(predictions.na.drop())

0.7103038747149434

In [40]:
metrics = RankingMetrics(predictions)

AttributeError: 'DataFrame' object has no attribute 'ctx'